# VIVOS TTS Training Notebook

Notebook này giúp huấn luyện mô hình TTS sử dụng dữ liệu VIVOS trên Google Colab với GPU.

## Cài đặt và chuẩn bị môi trường

Đầu tiên, clone repository từ GitHub và cài đặt các dependencies cần thiết.

In [1]:
# Kiểm tra GPU
!nvidia-smi

Thu Mar 27 08:36:21 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   67C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# Clone repository từ GitHub
!git clone https://github.com/DangAW2002/vietnamese-tts.git
%cd vietnamese-tts

Cloning into 'vietnamese-tts'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 17 (delta 1), reused 17 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (17/17), 10.87 KiB | 10.88 MiB/s, done.
Resolving deltas: 100% (1/1), done.
/content/vietnamese-tts


In [4]:
# Cài đặt các package cần thiết
!pip install torch torchaudio numpy==2.0.0 numba scipy librosa==0.10.1 matplotlib pandas tensorboard tqdm pyyaml soundfile gdown

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 97.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.7/253.7 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Tải dữ liệu đã được xử lý

Do kích thước lớn, dữ liệu VIVOS đã xử lý được lưu trữ riêng trên Google Drive. Chúng ta cần tải và giải nén trước khi sử dụng.

In [24]:
import os
import gdown
import zipfile

# URL của file zip chứa dữ liệu đã xử lý (cần thay thế bằng link Google Drive thật)
url = "https://drive.google.com/uc?id=1JT3DyD0wnz7-7i4Zf5D9Sp9Q90_iBsga"
zip_path = "vivos_processed.zip"

# Hỏi người dùng có muốn tải xuống không
download_data = input("Bạn đã cập nhật URL và muốn tải dữ liệu xuống? (yes/no): ")
if download_data.lower() in ["yes", "y"]:
    # Tải file zip
    print("Đang tải dữ liệu VIVOS đã xử lý...")
    gdown.download(url, zip_path, quiet=False, fuzzy=True)

    # Kiểm tra kích thước file
    if os.path.exists(zip_path):
        file_size = os.path.getsize(zip_path) / (1024 * 1024)  # Chuyển sang MB
        print(f"Kích thước file tải về: {file_size:.2f} MB")

        if file_size < 1:  # Kiểm tra nếu file quá nhỏ
            print("⚠ Cảnh báo: File tải xuống có kích thước bất thường, có thể bị lỗi!")

    # Giải nén
    print("Đang giải nén dữ liệu...")
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall("data")
        print("Hoàn thành tải và giải nén dữ liệu!")

        # Xóa file zip sau khi giải nén
        os.remove(zip_path)
    except zipfile.BadZipFile:
        print("❌ Lỗi: File tải xuống không phải là file ZIP hợp lệ!")
else:
    print("Bỏ qua bước tải dữ liệu. Hãy đảm bảo bạn đã có dữ liệu trong thư mục data/processed.")


Bạn đã cập nhật URL và muốn tải dữ liệu xuống? (yes/no): y
Đang tải dữ liệu VIVOS đã xử lý...


Downloading...
From (original): https://drive.google.com/uc?id=1JT3DyD0wnz7-7i4Zf5D9Sp9Q90_iBsga
From (redirected): https://drive.google.com/uc?id=1JT3DyD0wnz7-7i4Zf5D9Sp9Q90_iBsga&confirm=t&uuid=4e8bc0c0-55e7-4a86-bca4-be296e051df4
To: /content/vietnamese-tts/vivos_processed.zip
100%|██████████| 1.13G/1.13G [00:06<00:00, 172MB/s]


Kích thước file tải về: 1076.23 MB
Đang giải nén dữ liệu...
Hoàn thành tải và giải nén dữ liệu!


## Huấn luyện mô hình

Bây giờ chúng ta sẽ huấn luyện mô hình TTS sử dụng dữ liệu VIVOS đã được tiền xử lý.

In [23]:
# Kiểm tra dữ liệu đã được tải đúng cách chưa
import os
import glob

train_files = glob.glob("data/processed/train/*.pt")
val_files = glob.glob("data/processed/val/*.pt")
test_files = glob.glob("data/processed/test/*.pt")

print(f"Số file trong tập train: {len(train_files)}")
print(f"Số file trong tập val: {len(val_files)}")
print(f"Số file trong tập test: {len(test_files)}")

if len(train_files) == 0 or len(val_files) == 0 or len(test_files) == 0:
    print("CẢNH BÁO: Không tìm thấy dữ liệu đã xử lý. Hãy quay lại bước trước để tải dữ liệu.")

Số file trong tập train: 0
Số file trong tập val: 0
Số file trong tập test: 0
CẢNH BÁO: Không tìm thấy dữ liệu đã xử lý. Hãy quay lại bước trước để tải dữ liệu.


In [ ]:
# Chạy script training
# Các tham số có thể điều chỉnh:
# --epochs: số epoch
# --batch_size: kích thước batch
# --lr: learning rate

!python src/colab_train.py --epochs 50 --batch_size 64

## Hiển thị quá trình huấn luyện với TensorBoard

In [ ]:
# Cài đặt TensorBoard extension
%load_ext tensorboard
%tensorboard --logdir=checkpoints/logs

## Lưu mô hình đã huấn luyện lên Google Drive

Sau khi huấn luyện xong, bạn nên lưu mô hình lên Google Drive để tránh mất kết quả khi phiên Colab kết thúc.

In [ ]:
# Mount Google Drive để lưu mô hình
from google.colab import drive
drive.mount('/content/drive')

# Nén thư mục checkpoints
!zip -r checkpoints.zip checkpoints/

# Copy file nén vào Google Drive
import os
import shutil

# Tạo thư mục để lưu trữ mô hình trên Drive nếu chưa có
os.makedirs("/content/drive/My Drive/vietnamese-tts-models", exist_ok=True)

# Copy file zip vào Drive
shutil.copy("checkpoints.zip", "/content/drive/My Drive/vietnamese-tts-models/")
print("Đã lưu checkpoints.zip vào Google Drive thành công!")